In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/data/raw/CodeSearchNet')

sys.path.append('../src')

In [ ]:
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score

In [161]:
from datasets import load_dataset, Dataset

dataset = load_dataset("code_search_net", "python")

  0%|          | 0/3 [00:00<?, ?it/s]

In [163]:
repos_train = collections.Counter(dataset['train']['repository_name'])
repos_validation = collections.Counter(dataset['validation']['repository_name'])
repos_test = collections.Counter(dataset['test']['repository_name'])

In [148]:
def remove_params(example):
    example['func_documentation_string'] = re.sub('(.+):(.+)','\1', example['func_documentation_string']).replace('\n', '').replace('\x01', '')
    return example

In [158]:
import collections
frequencyDict = collections.Counter(dataset['repository_name'])

In [43]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

import importlib
#import preprocessing_utils
#importlib.reload(preprocessing_utils)

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
model_name = "SEBIS/code_trans_t5_large_source_code_summarization_python_multitask"
model=AutoModelWithLMHead.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model.to(device)
print(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


cuda
cuda


In [7]:
lengths = [len(x) for x in dataset['test']['func_documentation_string']]
enc_lengths = [len(x) for x in dataset['test']['func_code_string']]

In [8]:
np.array(lengths).max()
np.array(enc_lengths).mean()

1067.2319624819625

In [127]:
encoder_max_length = 124
decoder_max_length = 124


In [128]:
def batch_tokenize_preprocess(batch,
                              tokenizer,
                              max_source_length,
                              max_target_length):

    source = batch["func_code_string"]
    target = batch["func_documentation_string"]

    source_tokenized = tokenizer(
        source,
        padding="max_length",
        truncation=True,
        max_length=max_source_length
    )

    target_tokenized = tokenizer(
        target,
        padding="max_length",
        truncation=True,
        max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}

    # Ignore padding in the loss

    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]

    return batch


In [129]:
validation_data = dataset['validation'].map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=dataset['validation'].column_names,
)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:2792 in __getitem__            │
│                                                                                                  │
│   2789 │                                                                                         │
│   2790 │   def __getitem__(self, key):  # noqa: F811                                             │
│   2791 │   │   """Can be used to index columns (by string names) or rows (by integer index or i  │
│ ❱ 2792 │   │   return self._getitem(key)                                                         │
│   2793 │                                                                                         │
│   2794 │   def __getitems__(self, keys: List) -> List:                                           │
│   2795 │   │   """Can be used to get a batch using a list of integers indices."""                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:2776 in _getitem               │
│                                                                                                  │
│   2773 │   │   format_kwargs = kwargs["format_kwargs"] if "format_kwargs" in kwargs else self._  │
│   2774 │   │   format_kwargs = format_kwargs if format_kwargs is not None else {}                │
│   2775 │   │   formatter = get_formatter(format_type, features=self._info.features, **format_kw  │
│ ❱ 2776 │   │   pa_subtable = query_table(self._data, key, indices=self._indices if self._indice  │
│   2777 │   │   formatted_output = format_table(                                                  │
│   2778 │   │   │   pa_subtable, key, formatter=formatter, format_columns=format_columns, output  │
│   2779 │   │   )                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/formatting/formatting.py:580 in query_table     │
│                                                                                                  │
│   577 │   if not isinstance(key, (int, slice, range, str, Iterable)):                            │
│   578 │   │   _raise_bad_key_type(key)                                                           │
│   579 │   if isinstance(key, str):                                                               │
│ ❱ 580 │   │   _check_valid_column_key(key, table.column_names)                                   │
│   581 │   else:                                                                                  │
│   582 │   │   size = indices.num_rows if indices is not None else table.num_rows                 │
│   583 │   │   _check_valid_index_key(key, size)                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/formatting/formatting.py:520 in                 │
│ _check_valid_column_key                                                                          │
│                                                                                                  │
│   517                                                                                            │
│   518 def _check_valid_column_key(key: str, columns: List[str]) -> None:                         │
│   519 │   if key not in columns:                                                                 │
│ ❱ 520 │   │   raise KeyError(f"Column {key} not in the data

In [130]:
test_data = dataset['test'].map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=dataset['test'].column_names,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:2792 in __getitem__            │
│                                                                                                  │
│   2789 │                                                                                         │
│   2790 │   def __getitem__(self, key):  # noqa: F811                                             │
│   2791 │   │   """Can be used to index columns (by string names) or rows (by integer index or i  │
│ ❱ 2792 │   │   return self._getitem(key)                                                         │
│   2793 │                                                                                         │
│   2794 │   def __getitems__(self, keys: List) -> List:                                           │
│   2795 │   │   """Can be used to get a batch using a list of integers indices."""                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:2776 in _getitem               │
│                                                                                                  │
│   2773 │   │   format_kwargs = kwargs["format_kwargs"] if "format_kwargs" in kwargs else self._  │
│   2774 │   │   format_kwargs = format_kwargs if format_kwargs is not None else {}                │
│   2775 │   │   formatter = get_formatter(format_type, features=self._info.features, **format_kw  │
│ ❱ 2776 │   │   pa_subtable = query_table(self._data, key, indices=self._indices if self._indice  │
│   2777 │   │   formatted_output = format_table(                                                  │
│   2778 │   │   │   pa_subtable, key, formatter=formatter, format_columns=format_columns, output  │
│   2779 │   │   )                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/formatting/formatting.py:580 in query_table     │
│                                                                                                  │
│   577 │   if not isinstance(key, (int, slice, range, str, Iterable)):                            │
│   578 │   │   _raise_bad_key_type(key)                                                           │
│   579 │   if isinstance(key, str):                                                               │
│ ❱ 580 │   │   _check_valid_column_key(key, table.column_names)                                   │
│   581 │   else:                                                                                  │
│   582 │   │   size = indices.num_rows if indices is not None else table.num_rows                 │
│   583 │   │   _check_valid_index_key(key, size)                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/formatting/formatting.py:520 in                 │
│ _check_valid_column_key                                                                          │
│                                                                                                  │
│   517                                                                                            │
│   518 def _check_valid_column_key(key: str, columns: List[str]) -> None:                         │
│   519 │   if key not in columns:                                                                 │
│ ❱ 520 │   │   raise KeyError(f"Column {key} not in the data

In [ ]:
def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds, metrics_list=['rouge', 'bleu']):

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # POST PROCESSING
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    results_dict = {}
    for m in metrics_list:
        print(m)
        metric = evaluate.load(m)

        if m=='bleu':
            result = metric.compute(
              predictions=decoded_preds, references=decoded_labels,
              max_order=2

           )
        elif m=='rouge':
            result = metric.compute(
                predictions=decoded_preds, references=decoded_labels, use_stemmer=True
            )
        result = {key: value for key, value in result.items() if key!='precisions'}
        #result = {key: value * 100 for key, value in result.items() if 'rouge' in key}

        prediction_lens = [
            np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
        ]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        results_dict.update(result)
    print(results_dict)
    return results_dict


In [131]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["func_code_string"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [150]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=8,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=100,
    save_total_limit=3,
    report_to=None
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)



In [151]:
ttt = dataset.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=dataset.column_names,
)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [152]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    #train_dataset=train_data,
    eval_dataset=ttt,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



In [153]:
results = trainer.evaluate()

rouge
bleu
{'rouge1': 0.5586, 'rouge2': 0.47, 'rougeL': 0.5451, 'rougeLsum': 0.5477, 'gen_len': 11.24, 'bleu': 0.2496, 'brevity_penalty': 0.4019, 'length_ratio': 0.5231, 'translation_length': 1391, 'reference_length': 2659}


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    #train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

In [136]:
test_samples = dataset.select(range(20))

In [137]:
summaries_zero_shot = generate_summary(test_samples, model)[1]

In [139]:
for i, description in enumerate(test_samples["func_documentation_string"]):
  print('_'*10)
  print(i)
  print(f'Original: {description}')
  print(f'Summary before Tuning: {summaries_zero_shot[i]}')
  print('_'*10)
  print('\n')


__________
0
Original: Get a list of datetime sorted rows from a table in the database
 database connection
 name of table in the database
 optional uuid to pull from
 optional column/field in the table to pull instead of rows
 a list of tuples containing (datetime, row) pairs or (datetime, column) pairs if columns is specified.
Summary before Tuning: Get a list of datetime sorted rows from a table in the database
__________


__________
1
Original: Rename a table in the database
 database connection
 original table name
 new table name
Summary before Tuning: Rename a table in the database
__________


__________
2
Original: Create and return a logged in FTP object.

Summary before Tuning: Login to an hq server.
__________


__________
3
Original: Download the master database
 ftp connection
 path to file on the ftp server
 local path to download file

Summary before Tuning: Download the master database on the FTP server
__________


__________
4
Original: Setter for **sel

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=16,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=100,
    save_total_limit=3,
    report_to=None
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 21>:25                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_data' is not defined

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["input_sequence"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(20))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]